In [2]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster('local').setAppName('countbydate')
# year [0:4]
# month [5]
# day [7:9]
# hour [10:12]
# minute [13:15]
# second [16:18]
# AM or PM [19:21]
#transfer 12 hours to 24 hours format
def tranfer_pm(input_data):
    result = input_data.split(' ')
    date = result[0]
    time = result[1]
    tz = result[2]
    time_details = time.split(':')
    if(tz == 'PM'):
        time_details[0] = int(time_details[0])+12
        if (time_details[0] == 24):
            time_details[0] -= 24
        new_time = str(time_details[0]) + ':' + time_details[1] + ':' + time_details[2]
    else:
        new_time = time
    return date + ' ' + new_time

In [585]:
from pandas import DataFrame, Series
import pandas as pd

In [586]:
data = pd.read_csv('/Users/liuyang/Desktop/yellow_tripdata_2018-06.csv')

In [588]:
from dateutil.parser import parse
from datetime import datetime
import numpy as np
import pandas as pd
newData = data[['tpep_pickup_datetime', 'PULocationID']]
newData = newData[newData['PULocationID'] == 237]
newData['tpep_pickup_datetime'] = newData['tpep_pickup_datetime'].map(lambda x: parse(x).strftime('%Y/%m/%d %H:%M'))
newData = newData.sort_values(by = 'tpep_pickup_datetime', ascending = True)
newData['tpep_pickup_datetime'] = newData['tpep_pickup_datetime'].map(lambda x: datetime.strptime(x,'%Y/%m/%d %H:%M'))
cleaned_data = newData[(newData['tpep_pickup_datetime'] >= datetime.strptime('2018/6/1 0:0','%Y/%m/%d %H:%M'))&(newData['tpep_pickup_datetime'] < datetime.strptime('2018/7/1 0:0','%Y/%m/%d %H:%M'))]
cleaned_data
#37952 rows/ not consistent 
#len(newData['tpep_pickup_datetime'].value_counts())

,tpep_pickup_datetime,PULocationID
534,2018-06-01 00:00:00,237
3300,2018-06-01 00:00:00,237
11640,2018-06-01 00:00:00,237
2395,2018-06-01 00:00:00,237
9506,2018-06-01 00:00:00,237
...,...,...
8706170,2018-06-30 23:57:00,237
8706806,2018-06-30 23:57:00,237
8710749,2018-06-30 23:58:00,237
8708596,2018-06-30 23:58:00,237


In [589]:
# generate time series
test_date = pd.date_range(start='6/01/2018',end ='7/1/2018', freq='T')
test_date = pd.DataFrame(test_date)
test_date.rename(columns = {0:'pickup_time_g'}, inplace = True)

In [590]:
Counted_value = cleaned_data.groupby('tpep_pickup_datetime', as_index=False).count()
Counted_value.rename(columns={'PULocationID':'Count'}, inplace=True)
#test_date['pickup_time_g'][1] == Counted_value['tpep_pickup_datetime'][0]
matched_date = pd.merge(test_date, Counted_value, left_on = 'pickup_time_g', right_on = 'tpep_pickup_datetime', how = 'left')
matched_date['Count'] = matched_date['Count'].fillna(0)
matched_date.drop(columns=['tpep_pickup_datetime'],inplace = True)

In [591]:
def interval_extraction(dataset, window_num):
    Demand = pd.Series(matched_date['Count']).rolling(window = window_num).sum()
    # check the number of null values
    #tryit.isna().sum()
    Demand.dropna(inplace=True)
    Demand.reset_index(drop=True,inplace = True)
    dataset[f'Demand_{window_num}'] = Demand
    dataset.dropna(inplace = True)
    return dataset

In [593]:
Interval_15 = matched_date
dataset_15 = interval_extraction(Interval_15, 15)
dataset15

In [595]:
matched_date['weekday'] = matched_date['pickup_time_g'].dt.weekday

In [596]:
def separate_timestamp(matched_date):
    matched_date['month'] = matched_date['pickup_time_g'].map(lambda x: x.month)
    matched_date['hour'] = matched_date['pickup_time_g'].map(lambda x: x.hour)
    matched_date['minute'] = matched_date['pickup_time_g'].map(lambda x: x.minute)
    matched_date['second'] = matched_date['pickup_time_g'].map(lambda x: x.second)
    matched_date['day'] = matched_date['pickup_time_g'].map(lambda x: x.day)
    return matched_date

In [597]:
matched_date = separate_timestamp(matched_date)

In [599]:
location_name = ['Upper East Side South'] * len(matched_date)
matched_date['location'] = Series(location_name)
locationID = [237] * len(matched_date)
matched_date['locationID'] = Series(locationID)

In [600]:
final_result = matched_date[['pickup_time_g','month','day','hour','minute','Count','Demand_15','location','locationID','weekday']]

In [601]:
final_result.rename(columns = {'Count':'demand','Demand_15':'15minutes_demand','pickup_time_g':'tpep_pickup_datetime'},inplace = True)

/Users/liuyang/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [618]:
final_result.to_csv('/Users/liuyang/Desktop/Upper_East_Side_South.csv')

In [615]:
final_result.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            43177, 43178, 43179, 43180, 43181, 43182, 43183, 43184, 43185,
            43186],
           dtype='int64', length=43187)